# Dataframe validaciones diarias por localidad 2024

In [161]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import unicodedata

ds = 'troncal_2024.csv'
df = pd.read_csv(ds)
df2= df.copy()
df2.head()

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
0,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-10,19,DUAL,1
1,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-11,19,DUAL,7
2,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,17,DUAL,9
3,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,19,DUAL,5
4,(11359) 226A00-Pedagogica,4.65771,-74.058361,2024-01-12,20,DUAL,1


In [162]:
df.shape

(1628608, 7)

In [163]:
df.describe(include='all')

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
count,1628608,1.621489e+06,1.621489e+06,1628608,1.628608e+06,1628608,1.628608e+06
unique,718,NaN,NaN,366,NaN,2,NaN
top,(08000) Portal Tunal,NaN,NaN,2024-01-26,NaN,TRONCAL,NaN
freq,7923,NaN,NaN,4808,NaN,1090620,NaN
mean,NaN,4.647300e+00,-7.408770e+01,NaN,1.327862e+01,NaN,3.602492e+02
std,NaN,5.174985e-02,3.630330e-02,NaN,5.539746e+00,NaN,8.261560e+02
min,NaN,4.528831e+00,-7.420483e+01,NaN,0.000000e+00,NaN,1.000000e+00
25%,NaN,4.604883e+00,-7.411167e+01,NaN,9.000000e+00,NaN,1.700000e+01
50%,NaN,4.646547e+00,-7.408050e+01,NaN,1.300000e+01,NaN,1.010000e+02
75%,NaN,4.689507e+00,-7.406125e+01,NaN,1.800000e+01,NaN,3.580000e+02


In [164]:
df2["latitud"] = pd.to_numeric(df2["latitud"], errors="coerce")
faltan_lat = df2[df2["latitud"].isna()]
faltan_lat.head()

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
1614167,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,10,TRONCAL,376
1614168,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,11,TRONCAL,533
1614169,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,12,TRONCAL,561
1614170,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,13,TRONCAL,480
1614171,(50008) Corral Portal Dorado,NaN,NaN,2024-01-01,14,TRONCAL,504


In [165]:
#Para saber si todos los datos fantantes son de la misma estación
solo_una = faltan_lat['Estacion_Parada'].nunique() == 1
print(solo_una)   

True


In [166]:
estacion = "Corral Portal Dorado"     
nueva_lat = 4.680867
nueva_lon = -74.120859

# Para encontrar la parada en el df
est = df2['Estacion_Parada'].str.contains(estacion)

# Para rellenar donde actualmente hay NaN 
df2.loc[est & df2['latitud'].isna(), 'latitud'] = nueva_lat
df2.loc[est & df2['longitud'].isna(), 'longitud'] = nueva_lon

In [167]:
df2.describe(include="all")

,Estacion_Parada,latitud,longitud,fecha,hora,Sistema,validaciones
count,1628608,1.628608e+06,1.628608e+06,1628608,1.628608e+06,1628608,1.628608e+06
unique,718,NaN,NaN,366,NaN,2,NaN
top,(08000) Portal Tunal,NaN,NaN,2024-01-26,NaN,TRONCAL,NaN
freq,7923,NaN,NaN,4808,NaN,1090620,NaN
mean,NaN,4.647447e+00,-7.408785e+01,NaN,1.327862e+01,NaN,3.602492e+02
std,NaN,5.168408e-02,3.628985e-02,NaN,5.539746e+00,NaN,8.261560e+02
min,NaN,4.528831e+00,-7.420483e+01,NaN,0.000000e+00,NaN,1.000000e+00
25%,NaN,4.604883e+00,-7.411317e+01,NaN,9.000000e+00,NaN,1.700000e+01
50%,NaN,4.647050e+00,-7.408050e+01,NaN,1.300000e+01,NaN,1.010000e+02
75%,NaN,4.688957e+00,-7.406125e+01,NaN,1.800000e+01,NaN,3.580000e+02


In [168]:
FECHA= "fecha"
LATITUD= "latitud"
LONGITUD= "longitud"
VALIDACIONES = "validaciones"
ESTACION = "Estacion_Parada"  

NOMBRE_LOC = "LocNombre"

base = df2.copy()

#Shape de localidades
SHP_FILE = r"C:\Users\PaolaCastro\Maestría\Proyecto\Localidades\Loca.shp"

# Archivo de salida
OUT_CSV = "validaciones_localidad_diarias_2024.csv"
base = df2.copy()

# Tipos
base[FECHA] = pd.to_datetime(base[FECHA], errors="coerce")
base[LATITUD] = pd.to_numeric(base[LATITUD], errors="coerce")
base[LONGITUD] = pd.to_numeric(base[LONGITUD], errors="coerce")
base[VALIDACIONES] = pd.to_numeric(base[VALIDACIONES], errors="coerce")

# Para cargar localidades y CRS
gdf_loc = gpd.read_file(SHP_FILE)

# Como cartografía de Bogotá viene en EPSG:4686
if gdf_loc.crs is None:
    gdf_loc = gdf_loc.set_crs(4686, allow_override=True)
gdf_loc = gdf_loc.to_crs(4326)

# Puntos desde 'base' y a sistema métrico (EPSG:3116)
gdf = gpd.GeoDataFrame(base, geometry=gpd.points_from_xy(base[LONGITUD], base[LATITUD]), crs=4326)
gdf_m = gdf.to_crs(3116)
gdf_loc_m = gdf_loc.to_crs(3116)

# Para asignar los puntos que están claramente dentro de una localidad
est_dentro = gpd.sjoin(gdf_m, gdf_loc_m[[NOMBRE_LOC, "geometry"]], how="left", predicate="within")

con_loc = est_dentro[~est_dentro[NOMBRE_LOC].isna()].copy()             # puntos con localidad 
sin_loc = est_dentro[est_dentro[NOMBRE_LOC].isna()].drop(columns=[NOMBRE_LOC])  # sin localidad (a rescatar)

# Para los que quedan sin una, se le asigna a la localidad más cercana
tol_m = 25  # metros
near = gpd.sjoin_nearest(sin_loc, gdf_loc_m[[NOMBRE_LOC, "geometry"]], how="left", max_distance=tol_m, distance_col="dist", lsuffix="L", rsuffix="R")

# Si un punto está con más de una, se queda con la más cercana
near = (near.reset_index().sort_values(['index', 'dist']).drop_duplicates(subset='index', keep='first').set_index('index'))

# Combina los que están claramente y los que se asignaron por cercanía, y vuelve a 4326
combined_m = pd.concat([con_loc, near], axis=0).sort_index()

# Aseguramos que combined_m siga siendo un GeoDataFrame con CRS métrico (3116)
if not isinstance(combined_m, gpd.GeoDataFrame):
    combined_m = gpd.GeoDataFrame(combined_m, geometry="geometry", crs=3116)
elif combined_m.crs is None:
    combined_m = combined_m.set_crs(3116, allow_override=True)

combined = (combined_m.to_crs(4326).rename(columns={NOMBRE_LOC: "localidad"}).drop(columns=["dist", "index_right", "index_left"], errors="ignore"))

# Como Soacha no está en el SHP, procedo a sacar esos registros
soacha = {"(07503) SAN MATEO - C.C. UNISUR": "SOACHA", "(07504) TERREROS": "SOACHA", "(07505) LEON XIII": "SOACHA", "(07506) DESPENSA": "SOACHA"}
combined = combined[~combined[ESTACION].isin(soacha.keys())].copy()

# Agregación mensual por localidad 
valid_loc_dia = (combined.groupby(["fecha","localidad"], as_index=False)[VALIDACIONES].sum().rename(columns={VALIDACIONES: "validaciones_diarias"}).sort_values(["localidad", "fecha"]))

# Para consistencia de los nombres de localidad
mapeo = {'ENGATIVA': 'ENGATIVÁ','FONTIBON': 'FONTIBÓN','LOS MARTIRES': 'LOS MÁRTIRES','SAN CRISTOBAL': 'SAN CRISTÓBAL','USAQUEN': 'USAQUÉN','CIUDAD BOLIVAR': 'CIUDAD BOLÍVAR'}
valid_loc_dia['localidad'] = valid_loc_dia['localidad'].replace(mapeo)

#Para añadir la columna de zona
zonas = pd.read_csv("Localidades Bogotá por zona.csv")

# Crear llave en mayúsculas y sin espacios extra
zonas["loc_key"] = zonas["Localidad"].str.upper().str.strip()
valid_loc_dia["loc_key"] = valid_loc_dia["localidad"].str.upper().str.strip()

# Merge para traer la zona
valid_loc_dia = valid_loc_dia.merge(zonas[["loc_key", "Zona"]], on="loc_key", how="left")

valid_loc_dia = valid_loc_dia.drop(columns=["loc_key"])
valid_loc_dia = valid_loc_dia.rename(columns={"Zona": "zona"})

# Para que la columna zona esté justo después de localidad
cols = list(valid_loc_dia.columns)
cols.remove("zona")
idx_loc = cols.index("localidad")
cols = cols[:idx_loc + 1] + ["zona"] + cols[idx_loc + 1:]
valid_loc_dia = valid_loc_dia[cols]

# Exportación
valid_loc_dia.to_csv(OUT_CSV, index=False)
print(f"Exportado: {OUT_CSV}")
valid_loc_dia.head()

Exportado: validaciones_localidad_diarias_2024.csv


,fecha,localidad,zona,validaciones_diarias
0,2024-01-01,ANTONIO NARIÑO,Suroriental,4232
1,2024-01-02,ANTONIO NARIÑO,Suroriental,16051
2,2024-01-03,ANTONIO NARIÑO,Suroriental,19523
3,2024-01-04,ANTONIO NARIÑO,Suroriental,15834
4,2024-01-05,ANTONIO NARIÑO,Suroriental,15393


In [169]:
#Ahora, para exportar el CSV con las estaciones y su respectiva localidadd
est_loc = (combined[['Estacion_Parada', 'localidad']].drop_duplicates().sort_values(['localidad', 'Estacion_Parada']).reset_index(drop=True).rename(columns={ESTACION: "estación"}))
est_loc['localidad'] = est_loc['localidad'].replace(mapeo)

# Columna de zona
loc_zona = valid_loc_dia[["localidad", "zona"]].drop_duplicates()
est_loc = est_loc.merge(loc_zona, on="localidad", how="left")
cols = list(est_loc.columns)
idx_loc = cols.index("localidad")
cols = cols[:idx_loc + 1] + ["zona"] + cols[idx_loc + 1:]
cols.remove("zona")
est_loc = est_loc[cols]

#Exportación
est_loc.to_csv('estaciones_localidades_2024.csv', index=False, encoding='utf-8-sig')

In [170]:
#Ahora, quiero obtener las áreas por localidad
OUT_AREAS = "areas_por_localidad.csv"
gdf_loc_m["area_m2"] = gdf_loc_m.area

# Agrego por localidad (por si la misma localidad viene en varias filas)
areas_loc = (gdf_loc_m.assign(localidad=gdf_loc_m[NOMBRE_LOC].astype(str).str.strip()).groupby("localidad", as_index=False).agg(area_m2=("area_m2", "sum")))

areas_loc["area_ha"]  = (areas_loc["area_m2"] / 1e4).round(2)
areas_loc['localidad'] = areas_loc['localidad'].replace(mapeo)

#Para la columna de zonas
zonas["loc_key"] = zonas["Localidad"].str.upper().str.strip()
areas_loc["loc_key"] = areas_loc["localidad"].str.upper().str.strip()

areas_loc = areas_loc.merge(zonas[["loc_key", "Zona"]], on="loc_key", how="left")

areas_loc = areas_loc.drop(columns=["loc_key"])
areas_loc = areas_loc.rename(columns={"Zona": "zona"})

cols_areas = ["localidad", "zona", "area_m2", "area_ha"]
areas_loc = areas_loc[cols_areas]

# Para exportar:
areas_loc.to_csv(OUT_AREAS, index=False, encoding="utf-8-sig")